In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2
# from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [2]:
mannequin_path = os.path.join('../','eventsData/training/mannequin')
# not_mannequin_path = os.path.join('../','not_mannequin_filter')

mannequin_img_path = [os.path.join(mannequin_path,x) for x in os.listdir(mannequin_path)]
# not_mannequin_img_path = [os.path.join(not_mannequin_path,x) for x in os.listdir(not_mannequin_path)]

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

random.shuffle(mannequin_img_path)
random.shuffle(mannequin_img_path)
# random.shuffle(not_mannequin_img_path)
# random.shuffle(not_mannequin_img_path)

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

['../eventsData/training/mannequin/446.png', '../eventsData/training/mannequin/1458.png', '../eventsData/training/mannequin/1245.png', '../eventsData/training/mannequin/748.png', '../eventsData/training/mannequin/1716.png']
['../eventsData/training/mannequin/1590.png', '../eventsData/training/mannequin/1731.png', '../eventsData/training/mannequin/1676.png', '../eventsData/training/mannequin/560.png', '../eventsData/training/mannequin/1532.png']


In [3]:
train_mannequin_img, test_mannequin_img = train_test_split(mannequin_img_path,test_size = 0.20,random_state=42)

In [4]:
train_datagen = IDG(preprocessing_function=preprocess_input)
val_datagen = IDG(preprocessing_function=preprocess_input)
test_datagen = IDG(preprocessing_function=preprocess_input)

train_path = '../eventsData_train'
val_path = '../eventsData_val'
test_path = '../eventsData_test'
shape = (224,224,3)

In [5]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,seed=33)
val_generator = val_datagen.flow_from_directory(val_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,seed=33)
test_generator = test_datagen.flow_from_directory(test_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical')


Found 3109 images belonging to 2 classes.
Found 388 images belonging to 2 classes.
Found 390 images belonging to 2 classes.


In [6]:
efficientB2_model = MobileNetV2(include_top=False,weights='imagenet',input_shape=shape)
efficientB2_model.trainable=False
x = GlobalAveragePooling2D()(efficientB2_model.output)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2,activation='softmax')(x)

model = Model(efficientB2_model.input,x)
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [11]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

In [12]:
history=model.fit(train_generator,epochs=20,validation_data=val_generator)

Epoch 1/20
98/98 [==============================] - 5s 48ms/step - loss: 0.1612 - accuracy: 0.9344 - val_loss: 0.0359 - val_accuracy: 0.9897
Epoch 2/20
98/98 [==============================] - 4s 43ms/step - loss: 0.0393 - accuracy: 0.9875 - val_loss: 0.0220 - val_accuracy: 0.9923
Epoch 3/20
98/98 [==============================] - 4s 42ms/step - loss: 0.0274 - accuracy: 0.9920 - val_loss: 0.0160 - val_accuracy: 0.9974
Epoch 4/20
98/98 [==============================] - 4s 41ms/step - loss: 0.0200 - accuracy: 0.9932 - val_loss: 0.0134 - val_accuracy: 0.9974
Epoch 5/20
98/98 [==============================] - 4s 42ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.0119 - val_accuracy: 0.9948
Epoch 6/20
98/98 [==============================] - 4s 41ms/step - loss: 0.0133 - accuracy: 0.9971 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 7/20
98/98 [==============================] - 4s 42ms/step - loss: 0.0101 - accuracy: 0.9974 - val_loss: 0.0068 - val_accuracy: 0.9974
Epoch 8/20
98

In [13]:
model.save('mobilenetV2_softmax.h5')

In [10]:
# from tensorflow.keras.models import load_model
# model_import = load_model('mobilenetV2.h5')